In [1]:
from myMongoClient import *
from preparing_finetuning import *
import pandas as pd
%reload_ext autoreload
%autoreload 2

db = "documents-100039"
organisation_id = int(db[-6:])
mongo_client = MyMongoClient(db)

## Get a mapping form the file_name and the type of document it is

In [ ]:
label_versions = pd.read_csv("data/labels_results/2024-01-18_16:15:33_df_gold_labels.csv")
output = label_versions[label_versions['label_name'] == 'document type'][['doc_id', 'label_value']]
doc_hash_value = {rows['doc_id'] : rows['label_value'] for idx , rows in output.iterrows()}
file_name_doc_hash = {inst['files'][0]['fileName'] : inst['_id'] for inst in mongo_client.get_all_documents() if inst['files'] and inst['files'][0]['fileName'].startswith(('Pseudo_',"pseudo_"))}
file_nam_value = {key : doc_hash_value[value] for key, value in file_name_doc_hash.items() if value in doc_hash_value}
# file_nam_value, len(file_nam_value)

dump(file_nam_value, "doc_type_from_file_name.pkl")

### Verify that all label name exist in the mapping label_name:label_key

In [15]:
print_all_missing_mapping()

In [ ]:
def update_mapping_class_label_label_name(label_class, label_name):
    file = load("classifaction_label_to_label_name.pkl")
    file[label_class] = label_name
    dump(file, "classifaction_label_to_label_name.pkl") 

In [ ]:
all_docs_labels = pd.read_csv("data/labels_results/2024-01-12_16:42:23_df_gold_labels.csv")
def  get_label_value(file_name, label_name):
    doc = [doc for doc in all_docs_labels if doc['files'][0]['fileName'] == file_name]
    label = [label for label in doc['labels'] if label['name'] == label_name]
    return label['value'] if label else []
all_prompts = get_all_prompts()
all_prompts['fields_name_value'] = all_prompts.apply(lambda row : 
            {label_name : get_label_value(row['file_name'], label_name)for label_name in row['fields_name_key']}, axis = 1)

In [ ]:
user_llm = {
   "2vIzdtbIlRfELCEIj+Wsxw==" : '7wKQEgxKSZEspILT7n7UqA==',
   "4qvfu2lB0Qe9e6j0Kq+Q0Q==" : "3+h1BtgoAL0PhXWyK1+40g==",
   "HbCTvF8+qSdmSZx2554y1A==" : "VO580j3/ILBAvlhfKY7hFQ==",
   "0Dvjg/odKW5noqUL8wrOKw==" : "mHK8QQTTOTp0sx+85PF1qw==",
   "HwklrgHthb7XGEXVDgu6xg==" : "yEpfWSZXTPQZ84K+aj1drQ==",
   "HbCTvF8+qSdmSZx2554y1A==" : "VO580j3/ILBAvlhfKY7hFQ==",
   "hcqBfAlCslQ0a1QFu1RcJA==" : "63KSW0tjZ6CrwLEQoCfuUg==",
   "fTXgJLNmmgWMv9xzQjnRpQ==" : "29yradmnIewK1RIRbRSTrg==",
}
def map_doc_id_doc_name() :
    map_doc_id_doc_name = {}
    all_docs = mongo_client.get_all_documents()
    for doc_id in list(all_docs_labels['doc_id'].unique()):
        if doc_id in user_llm :
            doc_id = user_llm[doc_id] 
        map_doc_id_doc_name[doc_id] = [doc['files'][0]['fileName'] for doc in all_docs if doc['_id'] == doc_id and doc['files']][0]
    dump(map_doc_id_doc_name, "map_doc_id_doc_name.pkl")

In [14]:
all_prompts = get_all_prompts()
all_prompts

,messages,label,file_name,output,doc_id,fields_name,fields_key,fields_key_value
0,"[{'role': 'user', 'content': 'We are extractin...",client,131338_13p (1-4).pdf,{},5+ip+7sGl+kfXqh+5SwOyA==,"[client-client storage, client-type, client-no...","[client-client storage, client-type, client-no...",{}
1,"[{'role': 'user', 'content': 'We are finding i...",Type de document,131338_13p (1-4).pdf,{},5+ip+7sGl+kfXqh+5SwOyA==,[Type de document],[document type],{'document type': 'Acte'}
2,"[{'role': 'user', 'content': 'We are extractin...",client,131338_13p (5-13).pdf,{},DQmxo4NmXgSjMB1cYnqDLQ==,"[client-client storage, client-type, client-no...","[client-client storage, client-type, client-no...",{}
3,"[{'role': 'user', 'content': 'We are finding i...",Type de document,131338_13p (5-13).pdf,{},DQmxo4NmXgSjMB1cYnqDLQ==,[Type de document],[document type],{'document type': 'Division et réunion de parc...
4,"[{'role': 'user', 'content': 'We are extractin...",client,Pseudo_1002030 (11).pdf,{},xbv5g+6M8anyCzcHvk/aAw==,"[client-client storage, client-type, client-no...","[client-client storage, client-type, client-no...",{}
...,...,...,...,...,...,...,...,...
227,"[{'role': 'user', 'content': 'We are finding i...",Other,Pseudo_567108 (114-114).pdf,{},qDx3ca5KQq1tPNpeTenADg==,[Date],[relevant date],{'relevant date': '2023-10-23T00:00:00+00:00'}
228,"[{'role': 'user', 'content': 'We are finding i...",Other,Pseudo_567108 (115-116).pdf,{},TPC9ga0BcvDufyQG5rhJlA==,[Date],[relevant date],{'relevant date': '2012-08-08T00:00:00+00:00'}
229,"[{'role': 'user', 'content': 'We are finding i...",Other,Pseudo_567108 (117-117).pdf,{},NpG1ncW32zldaK0/WnAswQ==,[Date],[relevant date],{'relevant date': '2007-04-19T00:00:00+00:00'}
230,"[{'role': 'user', 'content': 'We are finding i...",Other,Pseudo_567108 (118-118).pdf,{},VblBUfdotuRXo6mkyE0zJA==,"[Date, Type de correspondance, Concerne, Signa...","[relevant date, correspondance type, invoice c...","{'relevant date': '2020-06-28T00:00:00+00:00',..."


In [7]:
load("data/labels_results/2023-11-01-gold_labels_filenames")

Error loading file data/labels_results/2023-11-01-gold_labels_filenames: Ran out of input


In [9]:
pd.read_csv("data/labels_results/2024-01-12_16:42:23_df_gold_labels.csv")['label_name'].unique()

array(['canton', 'client', 'code declaration', 'description document',
       'document type', 'numero contribuable', 'relevant date', 'year',
       'account number', 'annee salaire', 'mois salaire',
       'profession employé', 'salaire brut', 'salaire net',
       'revenu imposable icc', 'revenu imposable ifd',
       'fortune imposable icc', 'acte type', 'num parcelle', 'type plan',
       'bank', 'num avs', 'contract type', 'immeuble', 'montant loyer',
       'debut assurance', 'fin contrat', 'insurance name',
       'police number', 'prime annuelle', 'versement minimum garanti',
       'num dossier mutation', 'direction', 'invoice concern', 'owner',
       'sender', 'correspondance type', 'receiver'], dtype=object)